# Build clean 20x training set 
### what we have: a 200k+ 20x training set.  Removed white 256, classfied as tum and nor
### Fonctionalities: 
#### remove blurred 256
#### save in multiple 5k subsets 

In [1]:
import os
import numpy as np
import cv2

In [2]:
path_full = 'E:\\deeplearning\\Hepatocarcinomes\\data\\20x\\training\\split128_image\\nor_100k+'
path_subset = 'E:\\deeplearning\\Hepatocarcinomes\\data\\20x\\training\\split128_image\\pos_nor_5k'

In [3]:
limgs = []
for root, folders, files in os.walk(path_full, topdown=False):
    for name in files:
        if name.endswith('.tif'):
            limgs.append(root+'\\'+name)
# print(len(limgs))

In [4]:
#Edge Detection filter
kernel = np.array(([-1, -1, -1],
        [-1, 8, -1],
        [-1, -1, -1]), np.float32)
threshold_filter = 25 #a blurred one is likely to be 5, can also remove half background

In [5]:
nsubset = 0
os.mkdir(os.path.join(path_subset, str(nsubset).zfill(2)))

from tqdm import tqdm
pbar1 = tqdm(total=len(limgs), desc='patches', miniters=1, position=0)

for i in range(len(limgs)):
    pbar1.update()    
    img = cv2.cvtColor(cv2.imread(limgs[i], -1), cv2.COLOR_BGR2RGB)

    #if the subtile is not mostly blurred nor mostly over-colored, save them
    y = cv2.filter2D(img[:,:,:], -1, kernel)
    for a in range(y.shape[0]):
        for b in range(y.shape[1]):
            for c in range(y.shape[2]):
                if y[a,b,c] < 0:
                    y[a,b,c] = 0
                if y[a,b,c] > 255:
                    y[a,b,c] = 255
    m_filtered_sub = np.mean(y)

    if (m_filtered_sub > threshold_filter):
        n = len([x for x in os.listdir(os.path.join(path_subset, str(nsubset).zfill(2))) if x.endswith('.tif')])
        if (n==5000):
            nsubset = nsubset + 1
            n = 0
            os.mkdir(os.path.join(path_subset, str(nsubset).zfill(2)))
        n = str(n).zfill(4)
        cv2.imwrite(os.path.join(path_subset, str(nsubset).zfill(2), 'nor_'+str(nsubset).zfill(2)+'_'+n+'.tif'), cv2.cvtColor(img[:,:,:], cv2.COLOR_RGB2BGR))


# print(limg)
# print(llabel)

patches: 100%|██████████████████████| 150585/150585 [10:26:50<00:00,  3.76it/s]